# LOAD THE MODEL AND START USING IT

In [12]:
from CatGPT_model import GPT, GPTConfig
import torch
from tokenizers import ByteLevelBPETokenizer
import torch.nn.functional as F

In [71]:
def load_model(model, model_path, device='cpu'):
    """
    Load the model state dictionary from the specified path and load it into the model.

    Parameters:
    model (torch.nn.Module): The model into which the state dictionary will be loaded.
    model_path (str): The path from where the model will be loaded.
    device (torch.device): The device on which the model will be loaded.
    """
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    print(f"Model loaded from {model_path}")

model = GPT(GPTConfig(vocab_size=32768))
model = torch.compile(model)
load_model(model, model_path="../models/CatGPT.pth")

Model loaded from ../models/CatGPT.pth


In [14]:
def generate_text(input_text = 'La intel·ligència artificial tindrà la capactiat de', num_return_sequences = 1, max_length = 100, device='cpu'):

    enc = ByteLevelBPETokenizer(
        '../tokenizer/vocab.json',
        '../tokenizer/merges.txt'
    )

    # Encode the input text
    tokens = enc.encode(input_text).ids
    tokens = torch.tensor(tokens, dtype=torch.long)  # (8,)

    if len(tokens) > max_length:
        max_length = len(tokens) + 50
        print(f"Max length set to {max_length} as input text is longer than the previous max length")
    tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)  # (B, 8)
    x = tokens.to(device)

    # Set manual seed for reproducibility
    torch.manual_seed(42)

    # Generate sequences
    with torch.no_grad():
        for _ in range(max_length - x.size(1)):
            logits, _ = model(x)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, 1)
            x = torch.cat((x, next_token), dim=1)

    # Decode and print the generated sequences
    for i in range(num_return_sequences):
        tokens = x[i].tolist()
        decoded = enc.decode(tokens)
        print(f"Sample {i+1}: {decoded}")

### Examples

In [72]:
generate_text(input_text="El clima mediterrani permet gaudir de llargues jornades assolellades a la vora del mar.", num_return_sequences=3, max_length=50, device='cpu')

Sample 1: El clima mediterrani permet gaudir de llargues jornades assolellades a la vora del mar. Les reserves naturals de Figeres amaguen profundes llacunes mantenen les seves úniques platges terra endins. L'existència de grans extensions de bosc blanc a les riberes de
Sample 2: El clima mediterrani permet gaudir de llargues jornades assolellades a la vora del mar. Els hiverns estan entre els de plena hivernació quan les temperatures són altes i els de tipus calorós els cap als finals de primavera i estiu. Com per fer-ho
Sample 3: El clima mediterrani permet gaudir de llargues jornades assolellades a la vora del mar. Un alt poder estrés se situa al centre. Els estius són més llargs, i, a més, ofereixen un clima continentalitzat. Els hiverns —com tot el sector


In [24]:
generate_text(input_text="El arbres del bosc eren tenebrosos, un somriure es va escoltar i va ser quan", num_return_sequences=3, max_length=50)

Sample 1: El arbres del bosc eren tenebrosos, un somriure es va escoltar i va ser quan va arribar el moment del cant d’un tió. Pel matí es van acostar a entrar al bosc per veure els tions, que podia ser-ho humor
Sample 2: El arbres del bosc eren tenebrosos, un somriure es va escoltar i va ser quan al final ho vam aconseguir de nou en solitari. Aquesta Caputxeta Vermella viu a París i troba una estranya professora a la que no li interessa pels seus trets d'estu
Sample 3: El arbres del bosc eren tenebrosos, un somriure es va escoltar i va ser quan va tocar el violí. Un jugador amb l’esquerra com estava tocat va haver de parar perquè no es podia tocar. Al principi, els jugadors de Worces


In [25]:
generate_text(input_text="El sol es ponia en l'horitzó de milers de colors però els pirates no podien", num_return_sequences=3, max_length=50)

Sample 1: El sol es ponia en l'horitzó de milers de colors però els pirates no podien navegar per les aigües del Mar d'Aral. Volataren al volador terra.
Contes pràctics sobre la cuina de proximitat del CAVA,
Sample 2: El sol es ponia en l'horitzó de milers de colors però els pirates no podien anar. Ihota de na Pepita rodonaçant el cap d'un dorero, el cap d'una mare que vol una estimada per fer
Sample 3: El sol es ponia en l'horitzó de milers de colors però els pirates no podien desplaçar-se al Carib per a vigilar l'oest. Per això van haver de parar perquè no trobaven cap vora estreta que els permetés captar la llum pel verd


In [17]:
generate_text(input_text="La intel·ligència artificial tindrà la capacitat de", num_return_sequences=3, max_length=50)

Sample 1: La intel·ligència artificial tindrà la capacitat de dissenyar receptes per respondre a les preguntes que tothom pugui fer. Pel que fa a la indústria tecnològica, aquest argument el fem servir encara que no ho especifiquéssim. Així les empreses trobaran coneguts que les
Sample 2: La intel·ligència artificial tindrà la capacitat de saber quan ho dirà la matemàtica, en funció de si viu seguint viu o físicament viu. En el sentit més social, però, consumeixen una part d’allò que aconsegueixen mitjançant vídeo jocs propis (i
Sample 3: La intel·ligència artificial tindrà la capacitat de desplaçar a alguns perifèrics per a vigilar l'ús de recursos més complexos. La visió i el cervell objectiu de la recerca és un grau de disseny pel que fa a les capacitats mentals a mesura que el


In [21]:
generate_text(input_text="Els animals més perillosos del planeta terra són", num_return_sequences=3, max_length=50)

Sample 1: Els animals més perillosos del planeta terra són, sens dubte, els ximpanzés. Gairebé tots mantenen les seves colònies a terra ferma mentre segueixen el rastre del seu pas formant diferents espècies de tortugues, casi totes forçades i lligades a la força. La
Sample 2: Els animals més perillosos del planeta terra són al cel, per la qual cosa el cos no podia fer cap acció semblant per protegir el planeta, i cap animal que tingués el consum disparat o tingués la necessitat de consumir aigua neta de l'aixeta a
Sample 3: Els animals més perillosos del planeta terra són: a) Animals especials se solen classificar pel seu comportament com a més perillosos, i, perquè no, més voraços tenen comportaments nocius. —A tot el planeta les mascotes ni a tot el planeta és
